In [1]:
#|default_exp toolloop

# Tool Loop

The code for Claudette's tool loop should essentially work as is. We'll replicate the whole original notebook just to make sure

In [1]:
#| export

from gaspare.core import *
from fastcore.utils import *
from fastcore.meta import delegates

from google import genai

In [33]:
models

['gemini-2.0-flash-exp',
 'gemini-2.0-flash-exp-image-generation',
 'gemini-2.0-flash',
 'gemini-2.0-flash-001',
 'gemini-2.0-pro-exp-02-05',
 'gemini-2.0-flash-lite',
 'gemini-1.5-flash',
 'gemini-1.5-pro',
 'gemini-1.5-pro-002',
 'gemini-1.5-flash-8b',
 'gemini-2.0-flash-thinking-exp-01-21']

In [60]:
model = 'gemini-2.0-flash-lite'
model

'gemini-2.0-flash-lite'

Let's use the [same example](https://github.com/anthropics/anthropic-cookbook/blob/main/tool_use/customer_service_agent.ipynb) from Claudette's documentation.

In [61]:
orders = {
    "O1": dict(id="O1", product="Widget A", quantity=2, price=19.99, status="Shipped"),
    "O2": dict(id="O2", product="Gadget B", quantity=1, price=49.99, status="Processing"),
    "O3": dict(id="O3", product="Gadget B", quantity=2, price=49.99, status="Shipped")}

customers = {
    "C1": dict(name="John Doe", email="john@example.com", phone="123-456-7890",
               orders=[orders['O1'], orders['O2']]),
    "C2": dict(name="Jane Smith", email="jane@example.com", phone="987-654-3210",
               orders=[orders['O3']])
}

As with Claudette, we do not have to create the JSON schema manually. We can use docments.

In [62]:
def get_customer_info(
    customer_id:str # ID of the customer
): # Customer's name, email, phone number, and list of orders
    "Retrieves a customer's information and their orders based on the customer ID"
    print(f'- Retrieving customer {customer_id}')
    return customers.get(customer_id, "Customer not found")

def get_order_details(
    order_id:str # ID of the order
): # Order's ID, product name, quantity, price, and order status
    "Retrieves the details of a specific order based on the order ID"
    print(f'- Retrieving order {order_id}')
    return orders.get(order_id, "Order not found")

def cancel_order(
    order_id:str # ID of the order to cancel
)->bool: # True if the cancellation is successful
    "Cancels an order based on the provided order ID"
    print(f'- Cancelling order {order_id}')
    if order_id not in orders: return False
    orders[order_id]['status'] = 'Cancelled'
    return True

We are ready to go. The main difference here is that we don't assign the tools to the chat itself, since otherwise Gemini becomes too eager to use them.

In [54]:
tools = [get_customer_info, get_order_details, cancel_order]
chat = Chat(model)

In [63]:
pr = 'Can you tell me the email address for customer C1?'
pr = 'Tell me the email address for customer C1.'
r = chat(pr, tools=tools, use_afc=False)
r

- Retrieving customer C1


<ul><li><code>get_customer_info(customer_id=C1)</code></li></ul>
<details><ul><li><code>usage_metadata</code>: Cached: 0; In: 410; Out: 20; Total: 430</li><li><code>model_version</code>: gemini-2.0-pro-exp-02-05</li><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>index</code>: 0</li><li><code>finish_reason</code>: FinishReason.STOP</li><li><code>content</code>: <ul><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>function_call</code>: <ul><li><code>args</code>: <ul><li><b>customer_id</b>: C1</li></ul></li><li><code>name</code>: get_customer_info</li></ul></li></ul></details></li><li><code>role</code>: model</li></ul></li></ul></details></li><li><code>automatic_function_calling_history</code>: </li></ul></details>

In [64]:
chat()

The email address for customer C1 is john@example.com.<br />
<details><ul><li><code>usage_metadata</code>: Cached: 0; In: 332; Out: 14; Total: 346</li><li><code>model_version</code>: gemini-2.0-pro-exp-02-05</li><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>index</code>: 0</li><li><code>finish_reason</code>: FinishReason.STOP</li><li><code>content</code>: <ul><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: The email address for customer C1 is john@example.com.
</li></ul></details></li><li><code>role</code>: model</li></ul></li></ul></details></li><li><code>automatic_function_calling_history</code>: </li></ul></details>

In [67]:
sp = """You will be provided with tools, but don't limit your answer to those tools.
If the user query is related to some of the tools you have access to come up with a sequence of actions to achieve the goal and **execute the plan immediately**.

If the user query is unrelated to the tools you have access to, answer the query using your own knowledge."""

chat = Chat(model)
r = chat('Cancel all orders for customer C1.', tools=tools, use_afc=False, sp=sp, temp=0.6)
r

I am unable to fulfill this request. I can only cancel orders one at a time, and I need the order ID to do so.<br />
<details><ul><li><code>usage_metadata</code>: Cached: 0; In: 209; Out: 29; Total: 238</li><li><code>model_version</code>: gemini-2.0-flash-lite</li><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>finish_reason</code>: FinishReason.STOP</li><li><code>content</code>: <ul><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: I am unable to fulfill this request. I can only cancel orders one at a time, and I need the order ID to do so.
</li></ul></details></li><li><code>role</code>: model</li></ul></li><li><code>avg_logprobs</code>: -0.1087644100189209</li></ul></details></li><li><code>automatic_function_calling_history</code>: </li></ul></details>

In [58]:
#| exports

@patch
@delegates(genai.chats.Chat.__call__)
def toolloop(self:genai.chats.Chat,
             pr, # Prompt to pass to Gemini
             max_steps=10, # Maximum number of tool requests to loop through
             trace_func:Optional[callable]=None, # Function to trace tool use steps (e.g `print`)
             cont_func:Optional[callable]=noop, # Function that stops loop if returns False
             **kwargs):
    "Add prompt `pr` to dialog and get a response from Gemini, automatically following up with `tool_use` messages"
    n_msgs = len(self.h)
    kwargs["use_afc"] = False
    r = self(pr, **kwargs)
    for i in range(max_steps):
        if not r.function_calls:break
        if trace_func: trace_func(self.h[n_msgs:]); n_msgs = len(self.h)
        r = self(**kwargs)
        if not (cont_func or noop)(self.h[-2]): break
    if trace_func: trace_func(self.h[n_msgs:])
    return r

In [59]:
chat = Chat(model)
r = chat.toolloop('Tell me the email address for customer C1.', tools=tools, sp=sp, temp=0.)
r

- Retrieving customer C1


The email address for customer C1 is john@example.com.<br />
<details><ul><li><code>usage_metadata</code>: Cached: 0; In: 459; Out: 14; Total: 473</li><li><code>model_version</code>: gemini-2.0-pro-exp-02-05</li><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>index</code>: 0</li><li><code>finish_reason</code>: FinishReason.STOP</li><li><code>content</code>: <ul><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: The email address for customer C1 is john@example.com.
</li></ul></details></li><li><code>role</code>: model</li></ul></li></ul></details></li><li><code>automatic_function_calling_history</code>: </li></ul></details>

In [49]:
def print_msgs(msgs):
    for n, m in enumerate(msgs):
        for i, part in enumerate(m.parts):
            print(f"\nMessage {n+1}, Part {i + 1}:\n")
            c = "* Text *: " + part.text if part.text  else "" 
            c += "* Function Call *: " + str(part.function_call) if part.function_call else ""
            c += "* Function Response *: " + str(part.function_response.response['result']) if part.function_response else ""
            print(c)
            print()

chat = Chat(model)
r = chat.toolloop('Cancel all orders for customer C1.', tools=tools, trace_func=print_msgs, temp=0., sp=sp)
r

TypeError: 'NoneType' object is not iterable

GenerateContentResponse(candidates=[Candidate(content=Content(parts=None, role=None), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=None, finish_reason=<FinishReason.MALFORMED_FUNCTION_CALL: 'MALFORMED_FUNCTION_CALL'>, grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=None)], create_time=None, response_id=None, model_version='gemini-1.5-flash-8b-001', prompt_feedback=None, usage_metadata=Cached: 0; In: 209; Out: 0; Total: 209, automatic_function_calling_history=[], parsed=None)

In [50]:
chat.toolloop('What is the status of order O1?', tools=tools)

- Retrieving order O1


The order status is Shipped.<br />
<details><ul><li><code>usage_metadata</code>: Cached: 0; In: 161; Out: 7; Total: 168</li><li><code>model_version</code>: gemini-1.5-flash-8b-001</li><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>safety_ratings</code>: <details open='true'><summary>safety_ratings[0]</summary><ul><li><code>probability</code>: HarmProbability.NEGLIGIBLE</li><li><code>category</code>: HarmCategory.HARM_CATEGORY_HATE_SPEECH</li></ul></details>
<details open='true'><summary>safety_ratings[1]</summary><ul><li><code>probability</code>: HarmProbability.NEGLIGIBLE</li><li><code>category</code>: HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT</li></ul></details>
<details open='true'><summary>safety_ratings[2]</summary><ul><li><code>probability</code>: HarmProbability.NEGLIGIBLE</li><li><code>category</code>: HarmCategory.HARM_CATEGORY_HARASSMENT</li></ul></details>
<details open='true'><summary>safety_ratings[3]</summary><ul><li><code>probability</code>: HarmProbability.NEGLIGIBLE</li><li><code>category</code>: HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT</li></ul></details></li><li><code>finish_reason</code>: FinishReason.STOP</li><li><code>content</code>: <ul><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: The order status is Shipped.
</li></ul></details></li><li><code>role</code>: model</li></ul></li><li><code>avg_logprobs</code>: -0.09802218845912389</li></ul></details></li><li><code>automatic_function_calling_history</code>: </li></ul></details>

## Code interpreter

In [13]:
from toolslm.shell import get_shell
from fastcore.meta import delegates
import traceback

In [14]:
imps = 'os, warnings, time, json, re, math, collections, itertools, functools, dateutil, datetime, string, types, copy, pprint, enum, numbers, decimal, fractions, random, operator, typing, dataclasses'

def CodeChat(model: Optional[str] = None, ask:bool=True, tools=None, **kwargs):
    imps = 'os, warnings, time, json, re, math, collections, itertools, functools, dateutil, datetime, string, types, copy, pprint, enum, numbers, decimal, fractions, random, operator, typing, dataclasses'
    chat = Chat(model=model, **kwargs)
    chat.ask = ask
    chat.shell = get_shell()
    chat.shell.run_cell('import '+ imps)
    chat._tools = tools
    chat._tools.append(chat.run_code)
    return chat

In [15]:
@patch
def run_code(
    self:genai.chats.Chat,
    code:str,   # Code to execute in persistent IPython session
): # Result of expression on last line (if exists); '#DECLINED#' if user declines request to execute
    "Executes python code using a persistent IPython session. It asks the user for permission before executing the code."
    confirm = f'Press Enter to execute, or enter "n" to skip?\n```\n{code}\n```\n'
    if input(confirm): return '#DECLINED#'
    try: res = self.shell.run_cell(code)
    except Exception as e: return traceback.format_exc()
    return res.stdout if res.result is None else res.result

In [16]:
sp = f'''You are a knowledgable coding assistant assistant. 
Don't do complex calculations yourself -- create code for them. 
Whenever you create code, run it in the IPython environment using the `run_code` tool.
The following modules are pre-imported for `run_code` automatically:

{imps}

Note that `run_code` interpreter state is *persistent* across calls. 

If a tool returns `#DECLINED#` report to the user that the attempt was declined and no further progress can be made.'''

In [17]:
def get_user(
            ) -> str: # Username of current user
    """Get the username of the user running this session"""
    print("Looking up username")
    return 'Miko'

In [24]:
chat = CodeChat('gemini-2.0-flash-lite', tools=[get_user], sp=sp, ask=True, temp=0.6)

In [25]:
pr = '''Create a 1-line function `checksum` for a string `s`, that multiplies together the ascii 
values of each character in `s` using `reduce`.'''
r = chat.toolloop(pr, tools=chat._tools, sp=sp, temp=0.6)
r


Press Enter to execute, or enter "n" to skip?
```

from functools import reduce

def checksum(s: str) -> int:
    return reduce(lambda x, y: x * ord(y), s, 1)

```
 


```python<br />from functools import reduce<br /><br />def checksum(s: str) -> int:<br />    return reduce(lambda x, y: x * ord(y), s, 1)<br />```
<details><ul><li><code>usage_metadata</code>: Cached: 0; In: 310; Out: 41; Total: 351</li><li><code>model_version</code>: gemini-2.0-flash-lite</li><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>finish_reason</code>: FinishReason.STOP</li><li><code>content</code>: <ul><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: ```python
from functools import reduce

def checksum(s: str) -> int:
    return reduce(lambda x, y: x * ord(y), s, 1)
```</li></ul></details></li><li><code>role</code>: model</li></ul></li><li><code>avg_logprobs</code>: -0.050492170380383006</li></ul></details></li><li><code>automatic_function_calling_history</code>: </li></ul></details>

In [27]:
chat.toolloop("Use it to get the checksum of the username of this session.",  tools=chat._tools, sp=sp, temp=0.6)

Looking up username


Press Enter to execute, or enter "n" to skip?
```
from functools import reduce

def checksum(s: str) -> int:
    return reduce(lambda x, y: x * ord(y), s, 1)

print(checksum('Miko'))
```
 


The checksum of the username "Miko" is 96025545.<br />
<details><ul><li><code>usage_metadata</code>: Cached: 0; In: 511; Out: 21; Total: 532</li><li><code>model_version</code>: gemini-2.0-flash-lite</li><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>finish_reason</code>: FinishReason.STOP</li><li><code>content</code>: <ul><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: The checksum of the username "Miko" is 96025545.
</li></ul></details></li><li><code>role</code>: model</li></ul></li><li><code>avg_logprobs</code>: -0.005351128322737557</li></ul></details></li><li><code>automatic_function_calling_history</code>: </li></ul></details>